<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-&amp;-Tools" data-toc-modified-id="Import-&amp;-Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import &amp; Tools</a></span></li><li><span><a href="#Repnet-Model" data-toc-modified-id="Repnet-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Repnet Model</a></span></li><li><span><a href="#Load-Model" data-toc-modified-id="Load-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Model</a></span><ul class="toc-item"><li><span><a href="#Modify-Add-OP-Shape-Align(Dim-Tile)" data-toc-modified-id="Modify-Add-OP-Shape-Align(Dim-Tile)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Modify Add OP Shape Align(Dim Tile)</a></span></li><li><span><a href="#Simple-Conv3D-Model" data-toc-modified-id="Simple-Conv3D-Model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Simple Conv3D Model</a></span></li></ul></li><li><span><a href="#Frozen-Graph" data-toc-modified-id="Frozen-Graph-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Frozen Graph</a></span></li><li><span><a href="#Saved-Model" data-toc-modified-id="Saved-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saved Model</a></span><ul class="toc-item"><li><span><a href="#Modify-Transpose-Perm-Attributes(FusedBatchNormV3)" data-toc-modified-id="Modify-Transpose-Perm-Attributes(FusedBatchNormV3)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Modify Transpose Perm Attributes(FusedBatchNormV3)</a></span></li><li><span><a href="#Original-Model-Outputs-VS-Modified-Model-Outputs" data-toc-modified-id="Original-Model-Outputs-VS-Modified-Model-Outputs-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Original Model Outputs VS Modified Model Outputs</a></span></li><li><span><a href="#Onnx-Optimizer" data-toc-modified-id="Onnx-Optimizer-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Onnx Optimizer</a></span></li><li><span><a href="#Fix-Unsqueeze-Op-(axis-attr)-Error-(TF--->-Onnx--->-OpenVINO)" data-toc-modified-id="Fix-Unsqueeze-Op-(axis-attr)-Error-(TF--->-Onnx--->-OpenVINO)-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fix Unsqueeze Op (axis attr) Error (TF --&gt; Onnx --&gt; OpenVINO)</a></span></li><li><span><a href="#Repnet-Inference" data-toc-modified-id="Repnet-Inference-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Repnet Inference</a></span></li><li><span><a href="#Calculate-Count" data-toc-modified-id="Calculate-Count-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Calculate Count</a></span><ul class="toc-item"><li><span><a href="#Inv-Period-Confidence-Score" data-toc-modified-id="Inv-Period-Confidence-Score-5.6.1"><span class="toc-item-num">5.6.1&nbsp;&nbsp;</span>Inv-Period Confidence Score</a></span></li><li><span><a href="#Within-Period-Confience-Scores" data-toc-modified-id="Within-Period-Confience-Scores-5.6.2"><span class="toc-item-num">5.6.2&nbsp;&nbsp;</span>Within Period Confience Scores</a></span></li><li><span><a href="#Period-Count" data-toc-modified-id="Period-Count-5.6.3"><span class="toc-item-num">5.6.3&nbsp;&nbsp;</span>Period Count</a></span></li></ul></li></ul></li><li><span><a href="#TFlite" data-toc-modified-id="TFlite-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TFlite</a></span></li><li><span><a href="#References" data-toc-modified-id="References-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

## Import & Tools

In [3]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p ipywidgets,cv2,PIL,matplotlib,plotly
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard,tflite
%watermark -p onnx,onnxruntime,tensorrt,tvm
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests, shutil
import os.path as osp
import numpy as np

def _IMPORT(x):
    try:
        x = x.strip()
        if x.startswith('https://'):
            x = x[8:]
        if not x.endswith('.py'):
            x = x + '.py'
        if x[0] == '/':
            with open(x) as fr:
                x = fr.read()
        else:
            x = x.replace('blob/main/', '').replace('blob/master/', '')
            if x.startswith('raw.githubusercontent.com'):
                uri = 'https://' + x
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
            elif x.startswith('github.com'):
                uri = x.replace('github.com', 'raw.githubusercontent.com')
                mod = uri.split('/')
                for s in ['main', 'master']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[-3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
            elif x.startswith('gitee.com'):
                mod = x.split('/')
                for s in ['/raw/main/', '/raw/master/']:
                    uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:])
                    x = requests.get(uri)
                    if x.status_code == 200:
                        x = x.text
                        break
        exec(x, globals())
    except:
        pass

def _DIR(x, dumps=True, ret=True):
    attrs = sorted([y for y in dir(x) if not y.startswith('_')])
    result = '%s: %s' % (str(type(x))[8:-2], json.dumps(attrs) if dumps else attrs)
    if ret:
        return result
    print(result)



numpy 1.19.5
sklearn 0.0
pandas 1.1.5
ipywidgets 7.6.3
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
plotly 5.3.0
torch 1.8.1+cu101
torchvision 0.9.1+cu101
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
tflite not installed
onnx 1.10.1
onnxruntime 1.8.1
tensorrt not installed
tvm not installed


In [2]:
%watermark -p onnxsim

onnxsim 0.3.6


In [25]:
def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<center><video %s %s controls src="%s" type="video/mp4"/></center>' % (W, H, data_url))

In [11]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    try:
        netron.start(file, address=('0.0.0.0', port), browse=False)
    except:
        pass
    display_html(port, height)
    
@register_line_magic
def tensorboard(line):
    import signal, shlex
    from tensorboard import manager as tbmanager

    args = line.split()
    logdir, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    
    infos = tbmanager.get_all()
    for info in infos:
        if info.port != port: continue
        try:
            os.kill(info.pid, signal.SIGKILL)
            os.unlink(os.path.join(tbmanager._get_info_dir(), f'pid-{info.pid}.info'))
        except OSError as e:
            if e.errno != errno.ENOENT: raise
        except Exception:
            pass
        break

    strargs = f'--host 0.0.0.0 --port {port} --logdir {logdir} --reload_interval 10'
    command = shlex.split(strargs, comments=True, posix=True)
    tbmanager.start(command)
    display_html(port, height)

In [7]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.lite.python.util import run_graph_optimizations, get_grappler_config
import onnx
import onnx.helper as OH
import onnxruntime as rt
from onnxsim import simplify
from onnx.numpy_helper import to_array, from_array
import shutil

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
gpus

[]

## Repnet Model

Repnet模型转换为onnx进行推理时, 存在两个问题:

<div class="alert alert-warning">
    <li> 加法算子操作输入shape不一致问题(pos_encoding)</li>
    <li> 3D卷积(5-D)后进行BatchNormlization时, 不支持5D</li>
</div>

In [6]:
layers = tf.keras.layers
regularizers = tf.keras.regularizers

In [4]:


def flatten_sequential_feats(x, batch_size, seq_len):
    """Flattens sequential features with known batch size and seq_len."""
    x = tf.reshape(x, [batch_size, seq_len, -1])
    return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.

      q, k, v must have matching leading dimensions.
      k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
      The mask has different shapes depending on its type(padding or look ahead)
      but it must be broadcastable for addition.

      Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable
              to (..., seq_len_q, seq_len_k). Defaults to None.

      Returns:
        outputs: shape == (..., seq_len_q, depth_v)
        attention_weights: shape == (..., seq_len_q, seq_len_k)
      """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk.
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    # (..., seq_len_q, seq_len_k)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])


class MultiHeadAttention(tf.keras.layers.Layer):
    """Multi-headed attention layer."""

    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth)."""
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(
            q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(
            k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(
            v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(
            scaled_attention,
            perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(
            scaled_attention,
            (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
    """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
    """

    def __init__(self, d_model, num_heads, dff,
                 dropout_rate=0.1,
                 reorder_ln=False):
        super(TransformerLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

        self.reorder_ln = reorder_ln

    def call(self, x):
        inp_x = x

        if self.reorder_ln:
            x = self.layernorm1(x)

        # (batch_size, input_seq_len, d_model)
        attn_output, _ = self.mha(x, x, x, mask=None)
        attn_output = self.dropout1(attn_output)

        if self.reorder_ln:
            out1 = inp_x + attn_output
            x = out1
        else:
            # (batch_size, input_seq_len, d_model)
            out1 = self.layernorm1(x + attn_output)
            x = out1

        if self.reorder_ln:
            x = self.layernorm2(x)

        # (batch_size, input_seq_len, d_model)
        ffn_output = self.ffn(x)
        ffn_output = self.dropout2(ffn_output)

        if self.reorder_ln:
            out2 = out1 + ffn_output
        else:
            # (batch_size, input_seq_len, d_model)
            out2 = self.layernorm2(out1 + ffn_output)

        return out2


def pairwise_l2_distance(a, b):
    """Computes pairwise distances between all rows of a and all rows of b."""
    norm_a = tf.reduce_sum(tf.square(a), 1)
    norm_a = tf.reshape(norm_a, [-1, 1])
    norm_b = tf.reduce_sum(tf.square(b), 1)
    norm_b = tf.reshape(norm_b, [1, -1])
    dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
    return dist


def get_sims(embs, temperature):
    """Calculates self-similarity between batch of sequence of embeddings."""
    # batch_size = tf.shape(embs)[0]
    # seq_len = tf.shape(embs)[1]
    # print(tf.shape(embs))
    # embs = tf.reshape(embs, [batch_size, seq_len, -1])

    def _get_sims(embs):
        """Calculates self-similarity between sequence of embeddings."""
        dist = pairwise_l2_distance(embs, embs)
        sims = -1.0 * dist
        return sims

    # sims = tf.map_fn(_get_sims, embs)
    # QRS: intel vpu not support while_loop
    sims = tf.expand_dims(_get_sims(embs[0]), 0)
    sims /= temperature
    sims = tf.nn.softmax(sims, axis=-1)
    sims = tf.expand_dims(sims, -1)
    return sims


class ResnetPeriodEstimator(tf.keras.models.Model):
    """RepNet model."""

    def __init__(
        self,
        num_frames=64,
        image_size=112,
        base_model_layer_name='conv4_block3_out',
        temperature=13.544,
        dropout_rate=0.25,
        l2_reg_weight=1e-6,
        temporal_conv_channels=512,
        temporal_conv_kernel_size=3,
        temporal_conv_dilation_rate=3,
        conv_channels=32,
        conv_kernel_size=3,
        transformer_layers_config=((512, 4, 512),),
        transformer_dropout_rate=0.0,
        transformer_reorder_ln=True,
        period_fc_channels=(512, 512),
        within_period_fc_channels=(512, 512)):
        super(ResnetPeriodEstimator, self).__init__()

        # Model params.
        self.num_frames = num_frames
        self.image_size = image_size

        self.base_model_layer_name = base_model_layer_name

        self.temperature = temperature

        self.dropout_rate = dropout_rate
        self.l2_reg_weight = l2_reg_weight

        self.temporal_conv_channels = temporal_conv_channels
        self.temporal_conv_kernel_size = temporal_conv_kernel_size
        self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

        self.conv_channels = conv_channels
        self.conv_kernel_size = conv_kernel_size
        # Transformer config in form of (channels, heads, bottleneck channels).
        self.transformer_layers_config = transformer_layers_config
        self.transformer_dropout_rate = transformer_dropout_rate
        self.transformer_reorder_ln = transformer_reorder_ln

        self.period_fc_channels = period_fc_channels
        self.within_period_fc_channels = within_period_fc_channels

        # Base ResNet50 Model.
        base_model = tf.keras.applications.ResNet50V2(
            include_top=False, weights=None, pooling='max')
        self.base_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=base_model.get_layer(self.base_model_layer_name).output)

        # 3D Conv on k Frames
        self.temporal_conv_layers = [
            layers.Conv3D(self.temporal_conv_channels,
                          self.temporal_conv_kernel_size,
                          padding='same',
                          dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                          kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                          kernel_initializer='he_normal')]
        self.temporal_bn_layers = [layers.BatchNormalization()
                                   for _ in self.temporal_conv_layers]

        # Counting Module (Self-sim > Conv > Transformer > Classifier)
        self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                            self.conv_kernel_size,
                                            padding='same',
                                            activation=tf.nn.relu)

        channels = self.transformer_layers_config[0][0]
        self.input_projection = layers.Dense(
            channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
            activation=None)

        self.input_projection2 = layers.Dense(
            channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
            activation=None)

        length = self.num_frames
        self.pos_encoding = tf.compat.v1.get_variable(
            name='resnet_period_estimator/pos_encoding',
            shape=[1, length, 1],
            initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
        self.pos_encoding2 = tf.compat.v1.get_variable(
            name='resnet_period_estimator/pos_encoding2',
            shape=[1, length, 1],
            initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

        self.transformer_layers = []
        for d_model, num_heads, dff in self.transformer_layers_config:
            self.transformer_layers.append(
                TransformerLayer(d_model, num_heads, dff,
                                 self.transformer_dropout_rate,
                                 self.transformer_reorder_ln))

        self.transformer_layers2 = []
        for d_model, num_heads, dff in self.transformer_layers_config:
            self.transformer_layers2.append(
                TransformerLayer(d_model, num_heads, dff,
                                 self.transformer_dropout_rate,
                                 self.transformer_reorder_ln))

        # Period Prediction Module.
        self.dropout_layer = layers.Dropout(self.dropout_rate)
        num_preds = self.num_frames//2
        self.fc_layers = []
        for channels in self.period_fc_channels:
            self.fc_layers.append(layers.Dense(
                channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                activation=tf.nn.relu))
        self.fc_layers.append(layers.Dense(
            num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

        # Within Period Module
        num_preds = 1
        self.within_period_fc_layers = []
        for channels in self.within_period_fc_channels:
            self.within_period_fc_layers.append(layers.Dense(
                channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                activation=tf.nn.relu))
        self.within_period_fc_layers.append(layers.Dense(
            num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

    def call(self, x):
        # Ensures we are always using the right batch_size during train/eval.
        batch_size = tf.shape(x)[0]
        # Conv Feature Extractor.
        x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
        x = self.base_model(x) # (64, 7, 7, 1024)
        # h = tf.shape(x)[1]
        # w = tf.shape(x)[2]
        # c = tf.shape(x)[3]
        x = tf.reshape(x, [batch_size, -1, 7, 7, 1024])
# 
        # 3D Conv to give temporal context to per-frame embeddings. 
        # for bn_layer, conv_layer in zip(self.temporal_bn_layers,
        #                                 self.temporal_conv_layers):
        #     x = conv_layer(x)
        #     x = bn_layer(x)
        #     x = tf.nn.relu(x)
        # 
        x = self.temporal_conv_layers[0](x) # 1, 64, 7, 7, 512
        x = self.temporal_bn_layers[0](x)
        x = tf.nn.relu(x)
        #     
        # (1, 64, 7, 7, 512)
        # x = tf.reshape(x, (batch_size, self.num_frames, 7, 7, -1))
        # (1, 64, 512)
        x = tf.reduce_max(x, axis=[2, 3])
        
        # Reshape and prepare embs for output.
        # final_embs = x
# 
        # Get self-similarity matrix.
        x = get_sims(x, self.temperature)
        print(x.shape)
# 
        # 3x3 conv layer on self-similarity matrix.
        x = self.conv_3x3_layer(x)
        x = tf.reshape(x, [batch_size, self.num_frames, -1])
        within_period_x = x
# 
        # Period prediction.
        x = self.input_projection(x)
        x += self.pos_encoding
        # for transformer_layer in self.transformer_layers:
        #     x = transformer_layer(x)
        print(x.shape)
        x = self.transformer_layers[0](x)
        
        x = flatten_sequential_feats(x, batch_size, self.num_frames)
        # for fc_layer in self.fc_layers:
        #     x = self.dropout_layer(x)
        #     x = fc_layer(x)
        x = self.dropout_layer(x)
        x = self.fc_layers[0](x)
        x = self.fc_layers[1](x)
        x = self.fc_layers[2](x)
# 
        # Within period prediction.
        within_period_x = self.input_projection2(within_period_x)
        within_period_x += self.pos_encoding2
        # for transformer_layer in self.transformer_layers2:
        #     within_period_x = transformer_layer(within_period_x)
        within_period_x = self.transformer_layers2[0](within_period_x)
        
        within_period_x = flatten_sequential_feats(within_period_x, batch_size, self.num_frames)
        # for fc_layer in self.within_period_fc_layers:
        #     within_period_x = self.dropout_layer(within_period_x)
        #     within_period_x = fc_layer(within_period_x)
        within_period_x = self.dropout_layer(within_period_x)
        within_period_x = self.within_period_fc_layers[0](within_period_x)
        within_period_x = self.within_period_fc_layers[1](within_period_x)
        within_period_x = self.within_period_fc_layers[2](within_period_x)

        return x, within_period_x #, final_embs
    
model = ResnetPeriodEstimator()
test_outputs = model(tf.random.uniform((1,64,112,112,3)))
test_outputs[0].shape

(1, 64, 64, 1)
(1, 64, 512)


TensorShape([1, 64, 32])

## Load Model

### Modify Add OP Shape Align(Dim Tile)

In [27]:
model = ResnetPeriodEstimator()
print(model.pos_encoding.shape)
print(model.pos_encoding[0][0][:10])
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore('/data/pretrained/cv/repnet/ckpt-88').expect_partial()
print('-'*20)
model.pos_encoding = tf.tile(model.pos_encoding, multiples=[1, 1, 512])
model.pos_encoding2 = tf.tile(model.pos_encoding2, multiples=[1, 1, 512])
print(model.pos_encoding.shape)
print(model.pos_encoding[0][0][:10])
test_inputs = np.random.randn(1, 64, 112, 112, 3).astype(np.float32)
test_inputs_tensor = tf.convert_to_tensor(test_inputs)
test_outputs = model(test_inputs_tensor)

(1, 64, 1)
tf.Tensor([-0.00579794], shape=(1,), dtype=float32)
--------------------
(1, 64, 512)
tf.Tensor(
[0.02054866 0.02054866 0.02054866 0.02054866 0.02054866 0.02054866
 0.02054866 0.02054866 0.02054866 0.02054866], shape=(10,), dtype=float32)


In [28]:
test_outputs[0].shape, test_outputs[1].shape # , test_outputs[2].shape

(TensorShape([1, 64, 32]), TensorShape([1, 64, 1]))

In [29]:
model.summary()

Model: "resnet_period_estimator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Functional)         (None, None, None, 1024)  5209600   
_________________________________________________________________
conv3d_2 (Conv3D)            multiple                  14156288  
_________________________________________________________________
batch_normalization_2 (Batch multiple                  2048      
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  320       
_________________________________________________________________
dense_40 (Dense)             multiple                  1049088   
_________________________________________________________________
dense_41 (Dense)             multiple                  1049088   
_________________________________________________________________
transformer_layer_4 (Transfo multiple    

### Simple Conv3D Model

tool | version
---: | :---:
tensorflow | 2.6.0
tensorboard | 2.6.0
onnx | 1.10.1
onnxruntime | 1.8.1
tf2onnx | 1.9.2

<div class="alert alert-warning">
针对上面版本, 转换onnx格式时, BatchNormalization会和Transpose操作融合为FusedBatchNormV3, Transpose的dim参数让人疑惑
</div>

In [16]:
@register_line_magic
def netron(line):
    if not line or line.strip() == 'help':
        print('%netron file port [height]')
        return
    args = line.split()
    file, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    try:
        netron.start(file, address=('0.0.0.0', port), browse=False)
    except:
        pass
    display_html(port, height)

In [8]:
model3d = tf.keras.models.Sequential([
    layers.Conv3D(
        512, 3, padding='same',
        dilation_rate=(3, 1, 1),
        kernel_regularizer=regularizers.l2(1e-6),
        kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.ReLU()
])

inputs3d = np.random.randn(1, 64, 7, 7, 1024).astype(np.float32)
outputs3d = model3d(inputs3d)

In [9]:
graph_opt = False
full_model = tf.function(lambda x: model3d(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(inputs3d.shape, inputs3d.dtype))
frozen_func = convert_variables_to_constants_v2(full_model, lower_control_flow=False)

frozen_graph_def = frozen_func.graph.as_graph_def()

frozen_graph_proto_path = '/data/nb_data/saved_models/'
as_text = False
frozen_graph3d_file = tf.io.write_graph(
    graph_or_graph_def=frozen_graph_def,
    logdir=frozen_graph_proto_path,
    name='frozen_graph_3d%s.pb%s' % ('_opt' if graph_opt else '', 'txt' if as_text else ''),
    as_text=as_text)
frozen_graph3d_file 

'/data/nb_data/saved_models/frozen_graph_3d.pb'

In [19]:
%netron {frozen_graph_file} 8999 500

In [116]:
saved_model3d_path = '/data/nb_data/saved_models'
model3d.save(saved_model3d_path, save_format='tf', include_optimizer=True)

INFO:tensorflow:Assets written to: /data/nb_data/saved_models/assets


In [ ]:
!python3 -m tf2onnx.convert --opset 13 \
    --tag serve --signature_def serving_default \
    --saved-model {saved_model3d_path} --output {saved_model3d_path}/saved_model3d.onnx 

In [ ]:
%netron {saved_model3d_path}/saved_model3d.onnx 8131 400

In [ ]:
sess = rt.InferenceSession(f'{saved_model3d_path}/saved_model3d.onnx')  
inputs = {sess.get_inputs()[0].name: inputs3d}
outputs = [o.name for o in sess.get_outputs()]
predictions = sess.run(outputs, inputs)
predictions[-1].shape, outputs3d[-1].shape

## Frozen Graph

In [132]:
graph_opt = False
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(test_inputs.shape, test_inputs.dtype))
frozen_func = convert_variables_to_constants_v2(full_model, lower_control_flow=False) # lower_control_flow == True -> error "unused_control_xxx"

'/data/nb_data/saved_models/frozen_graph.pb'

In [69]:
_DIR_(frozen_func), '-' * 60, \
frozen_func.inputs, frozen_func.outputs 

type: <class 'tensorflow.python.eager.wrap_function.WrappedFunction'>


('["add_gradient_functions_to_graph", "add_to_graph", "captured_inputs", "function_def", "graph", "inputs", "name", "output_dtypes", "output_shapes", "outputs", "pretty_printed_signature", "prune", "structured_input_signature", "structured_outputs", "trainable_variables", "variables"]',
 '------------------------------------------------------------',
 [<tf.Tensor 'x:0' shape=(1, 64, 112, 112, 3) dtype=float32>],
 [<tf.Tensor 'Identity:0' shape=(64, 7, 7, 1024) dtype=float32>])

In [ ]:
_DIR_(frozen_func.graph), '-' * 60, \
frozen_func.graph.inputs, frozen_func.graph.outputs

In [17]:
operations = frozen_func.graph.get_operations()
print(operations[590])
_DIR_(operations[590])

name: "resnet_period_estimator_4/transformer_layer_8/multi_head_attention_8/strided_slice"
op: "StridedSlice"
input: "resnet_period_estimator_4/transformer_layer_8/multi_head_attention_8/Shape"
input: "resnet_period_estimator_4/transformer_layer_8/multi_head_attention_8/strided_slice/stack"
input: "resnet_period_estimator_4/transformer_layer_8/multi_head_attention_8/strided_slice/stack_1"
input: "resnet_period_estimator_4/transformer_layer_8/multi_head_attention_8/strided_slice/stack_2"
attr {
  key: "Index"
  value {
    type: DT_INT32
  }
}
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "begin_mask"
  value {
    i: 0
  }
}
attr {
  key: "ellipsis_mask"
  value {
    i: 0
  }
}
attr {
  key: "end_mask"
  value {
    i: 0
  }
}
attr {
  key: "new_axis_mask"
  value {
    i: 0
  }
}
attr {
  key: "shrink_axis_mask"
  value {
    i: 1
  }
}

type: <class 'tensorflow.python.framework.ops.Operation'>


'["colocation_groups", "control_inputs", "device", "get_attr", "graph", "inputs", "name", "node_def", "op_def", "outputs", "run", "traceback", "type", "values"]'

In [ ]:
for i, op in enumerate(operations, 1):
    if op.name.startswith('unused'):
        print('%5d' % i, op.name)

In [62]:
graph_opt = True

input_tensors = [x for x in frozen_func.inputs if x.dtype != tf.resource]
output_tensors = frozen_func.outputs

if graph_opt:
    config = [
        'pruning', 'function', 'constfold', 'shape', 'remap', 'memory', 'loop',
        'common_subgraph_elimination', 'arithmetic', 'dependency', 'debug_stripper'
    ]

    frozen_graph_def = run_graph_optimizations(
        frozen_graph_def,
        input_tensors,
        output_tensors,
        config=get_grappler_config(config),
        graph=frozen_func.graph
    )

In [99]:
frozen_graph_proto_path = '/data/nb_data/saved_models/'
as_text = False
frozen_graph_file = tf.io.write_graph(
    graph_or_graph_def=frozen_graph_def,
    logdir=frozen_graph_proto_path,
    name='frozen_graph%s.pb%s' % ('_opt' if graph_opt else '', 'txt' if as_text else ''),
    as_text=as_text)
frozen_graph_file 

'/data/nb_data/saved_models/frozen_graph.pb'

In [104]:
%netron {frozen_graph_file} 8120 500

Stopping http://0.0.0.0:8120
Serving '/data/nb_data/saved_models/frozen_graph.pb' at http://0.0.0.0:8120


In [ ]:
inputs = ','.join([x.name for x in frozen_func.inputs])
outputs = ','.join([x.name for x in frozen_func.outputs])

!python3 -m tf2onnx.convert --opset 14 --input {frozen_graph_file} \
    --inputs {inputs} --outputs {outputs} --output {frozen_graph_file}.onnx

In [ ]:
%netron {frozen_graph_file}.onnx 8132

In [ ]:
onnx_model = onnx.load(f'{frozen_graph_file}.onnx')
try:
    onnx.checker.check_model(onnx_model, full_check=True) 
except Exception as err:
    print(err)
graph_def = onnx_model.graph
_DIR_(graph_def), '-'*60, graph_def.input, '-'*60, graph_def.output

In [ ]:
for i, node in enumerate(graph_def.node[95:99], 95):
    print('%5d' % i, node)

## Saved Model

In [17]:
saved_model_path = '/data/nb_data/saved_models'

In [18]:
model.save(saved_model_path, save_format='tf', include_optimizer=True)

INFO:tensorflow:Assets written to: /data/nb_data/saved_models_/assets


INFO:tensorflow:Assets written to: /data/nb_data/saved_models_/assets
/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [19]:
logdir=f'{saved_model_path}/log'
model_tf = tf.saved_model.load(saved_model_path)
sig = model_tf.signatures["serving_default"]
shutil.rmtree(logdir, ignore_errors=True)
writer = tf.summary.create_file_writer(logdir)
with writer.as_default():
    tf.summary.graph(sig.graph)

In [20]:
for node in sig.graph.as_graph_def().node[:3]:
    print(node.name)

input_1
unknown
unknown_0


In [21]:
%tensorboard {logdir} 8421 1000

In [ ]:
# !python3 -m tf2onnx.convert --help

In [22]:
!python3 -m tf2onnx.convert --opset 13 \
    --tag serve --signature_def serving_default \
    --saved-model {saved_model_path} --output {saved_model_path}/saved_model_op13_foldconst_dynamic.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-10-08 14:53:03,937 - INFO - Signatures found in model: [serving_default].
2021-10-08 14:53:03,939 - INFO - Output names: ['output_1', 'output_2']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-08 14:53:14,951 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-10-08 14:53:20,676 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-10-08 14:53:20,676 - INFO - Using opset <onnx, 13>
2021-10-08 14:53:48,791 - INFO - Computed 2 values for constant folding
2021-10-08 

In [ ]:
!python3 -m tf2onnx.convert --opset 13 --inputs input_1:0[1,64,112,112,3] \
    --tag serve --signature_def serving_default \
    --saved-model {saved_model_path} --output {saved_model_path}/saved_model_op13_foldconst.onnx 

In [23]:
# %netron {saved_model_path}/saved_model_op13_foldconst.onnx 8133 500
%netron {saved_model_path}/saved_model_op13_foldconst_dynamic.onnx 8133 500

Serving '/data/nb_data/saved_models_/saved_model_op13_foldconst_dynamic.onnx' at http://0.0.0.0:8133


### Modify Transpose Perm Attributes(FusedBatchNormV3)

In [ ]:
onnx_filename_prefix = 'saved_model_op13_foldconst_dynamic'

onnx_model = onnx.load(f'{saved_model_path}/{onnx_filename_prefix}.onnx')
onnx_model = onnx.shape_inference.infer_shapes(onnx_model, check_type=True) # for value_info
onnx_graph_def = onnx_model.graph
for i, node in enumerate(onnx_graph_def.node[100:200], 100):
    # print('%5d' % i, node.name)
    if 'FusedBatchNormV3' in node.name and 'Transpose' == node.op_type:
        print('-'*60)
        print(node)
        print('+'*60)
        attr = node.attribute.pop()
        if attr.ints[1] == 3:
            node.attribute.append(OH.make_attribute('perm', [0, 4, 1, 2, 3]))
        elif attr.ints[1] == 2:
            node.attribute.append(OH.make_attribute('perm', [0, 2, 3, 4, 1]))
        print('+'*60)
        print(node)
        print('-'*60)
        
onnx.save(onnx_model, f'{saved_model_path}/${onnx_filename_prefix}_mod.onnx')

In [ ]:
onnx_model.graph.node[0].input[0], onnx_model.graph.input

In [ ]:
len(onnx_graph_def.node), len(onnx_graph_def.value_info), len(onnx_graph_def.initializer)

In [ ]:
target_idx = 189
onnx_graph_def.value_info[target_idx], '-'*80, onnx_graph_def.node[target_idx]

In [ ]:
for i, w in enumerate(onnx_graph_def.initializer):
    if onnx_graph_def.node[target_idx].input[0][:-2] in w.name:
        print(w.dims, w.name)

In [ ]:
for i, proto in enumerate(onnx_model.graph.value_info):
    if 'ExpandDims' in proto.name:
        print('%5d' % i, proto)

In [ ]:
for i, node in enumerate(onnx_graph_def.node):
    if 'ExpandDims' in node.name:
        print('%5d'%i, node)

In [ ]:
# for i, node in enumerate(onnx_graph_def.node):
#     print(node.name)
    
for i, proto in enumerate(onnx_model.graph.value_info):
    print(proto)

In [ ]:
%netron {saved_model_path}/{onnx_filename_prefix}_mod.onnx 8134 600

### Original Model Outputs VS Modified Model Outputs

In [ ]:
# sess = rt.InferenceSession(f'{saved_model_path}/saved_model_op13_foldconst_mod.onnx')  
sess = rt.InferenceSession(f'{saved_model_path}/saved_model_op13_foldconst_dynamic_mod.onnx')  
inputs = {sess.get_inputs()[0].name: test_inputs}
outputs = [o.name for o in sess.get_outputs()]
outputs_onnx = sess.run(outputs, inputs)
outputs_onnx[-1].shape

In [ ]:
test_inputs.shape, sess.get_inputs()[0].name

In [ ]:
test_outputs[0][0][0][:3].numpy(), '-'*60, outputs_onnx[0][0][0][:3]

In [ ]:
test_outputs[1][0][:3].numpy().T, '-'*60, outputs_onnx[1][0][:3].T

### Onnx Optimizer

In [ ]:
onnx_model = onnx.load(f'{saved_model_path}/saved_model_op13_foldconst_mod.onnx')  
model_simp, check = simplify(
    onnx_model, dynamic_input_shape=False, 
    input_shapes={sess.get_inputs()[0].name: [1, 64, 112, 112, 3]})
if not check:
    raise
model_simp.graph.input, model_simp.graph.output

### Fix Unsqueeze Op (axis attr) Error (TF --> Onnx --> OpenVINO)

Run the script will crash, error info as below.

```bash
#!/bin/bash

CUR_DIR=$(cd $(dirname ${BASH_SOURCE[0]}); pwd)

EXEBIN=/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_onnx.py

$EXEBIN --input_model $CUR_DIR/saved_model_simp.onnx  --output_dir $CUR_DIR \
    --model_name repnet --input_shape [1,64,112,112,3] \
    --log_level DEBUG
```

<font color="red"><B>Error Info:</B></font>

```
[ 2021-09-03 18:45:59,828 ] [ DEBUG ] [ infer:117 ]  Partial infer for StatefulPartitionedCall/resnet_period_estimator/ExpandDims
[ 2021-09-03 18:45:59,828 ] [ DEBUG ] [ infer:118 ]  Op: ExpandDims
[ 2021-09-03 18:45:59,828 ] [ DEBUG ] [ infer:119 ]  Inputs:
[ 2021-09-03 18:45:59,828 ] [ DEBUG ] [ infer:19 ]  input[0]: shape = [ 1 64 64], value = <UNKNOWN>
[ 2021-09-03 18:45:59,829 ] [ DEBUG ] [ infer:19 ]  input[1]: shape = [1], value = [-1]
[ ERROR ]  Cannot infer shapes or values for node "StatefulPartitionedCall/resnet_period_estimator/ExpandDims".
[ ERROR ]  Wrong number of inputs to the layer StatefulPartitionedCall/resnet_period_estimator/ExpandDims
[ ERROR ]  
[ ERROR ]  It can happen due to bug in custom shape infer function <function ExpandDims.infer at 0x7f16e38afdc0>.
[ ERROR ]  Or because the node inputs have incorrect values/shapes.
[ ERROR ]  Or because input shapes are incorrect (embedded to the model or passed via --input_shape).
```

<div class="alert alert-danger">
IF dynamic_input_shape is true (input batch size is dynamic), the mo_onnx will crash. 
</div>

```    
[ ERROR ]  -------------------------------------------------
[ ERROR ]  ----------------- INTERNAL ERROR ----------------
[ ERROR ]  Unexpected exception happened.
[ ERROR ]  Please contact Model Optimizer developers and forward the following information:
[ ERROR ]  Exception occurred during running replacer "REPLACEMENT_ID (<class 'extensions.middle.ApplyPermutations.ApplyPermutation'>)": Cannot infer `StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/SpaceToBatchND_transpose__511` due to both order and reverse_order was set
```

In [ ]:
# for node in onnx_model.graph.node:
#     if 'conv3d/Conv3D/SpaceToBatchND_cast__479' in node.name or \
#     'required_space_to_batch_paddings' in node.name:
#         print(node)

In [ ]:
# model_simp.graph.value_info

In [ ]:
# for weight in onnx_model.graph.initializer:
#     if 'conv3d/Conv3D/required_space_to_batch_paddings/paddings_Concat__462' in weight.name:
#         print(weight)

In [ ]:
model_simp_ = copy.deepcopy(model_simp)
for i, node in enumerate(model_simp_.graph.node):
    if 'ExpandDims' in node.name or node.op_type == 'Unsqueeze':
        for j, weight in enumerate(model_simp_.graph.initializer):
            for x in node.input:
                if x.startswith('const_') and x in weight.name and weight.data_type == onnx.TensorProto.INT64:
                    tensor_value = to_array(weight)
                    for attr in node.attribute:
                        if attr.name == 'axes':
                            node.attribute.remove(attr)
                            break
                    node.input.remove(x)
                    node.attribute.append(OH.make_attribute('axes', tensor_value))
                    print('(%d,%d)\n%s\n'%(i, j, '-'*60))
                    print(node, weight, tensor_value, type(tensor_value[0]), '\n')
                    break

In [ ]:
for i, node in enumerate(model_simp_.graph.node):
    if 'Sum' in node.name:
        print(node)

In [ ]:
# model_simp.CopyFrom(model_simp_)
onnx.save(model_simp, f'{saved_model_path}/saved_model_op13_foldconst_mod_simp.onnx')

In [ ]:
%netron {saved_model_path}/saved_model_op13_foldconst_mod_simp.onnx 8135 500

In [ ]:
# sess = rt.InferenceSession(f'{saved_model_path}/saved_model_op13_foldconst_mod_simp.onnx')  
# inputs = {sess.get_inputs()[0].name: test_inputs}
# outputs = [o.name for o in sess.get_outputs()]
# outputs_onnx = sess.run(outputs, inputs)
# outputs_onnx[-1].shape

### Repnet Inference

In [ ]:
import cv2
from scipy.signal import medfilt
from scipy.special import softmax

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

num_frames = 64
# video_path = '/data/testvids/test1.mp4'
video_path = 'https://frepai.s3.didiyunapi.com/datasets/test/repnet_bird.gif'

def read_video(video_filename, width=112, height=112):
    cap = cv2.VideoCapture(video_filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    if cap.isOpened():
        while True:
            success, frame_bgr = cap.read()
            if not success:
                break
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.resize(frame_rgb, (width, height))
            frames.append(frame_rgb)
        frames = np.asarray(frames)
    return frames, fps

video_frames, fps = read_video(video_path)

In [ ]:
# show_image(video_path)

In [ ]:
seq_len = len(video_frames)
diff = math.ceil(seq_len / num_frames) * num_frames - seq_len
fps, seq_len, diff

In [ ]:
# align num_frames
frames = np.take(video_frames, indices=list(range(seq_len)) + [seq_len-1] * diff, axis=0)
batch_size = int(len(frames) / num_frames)
len(frames), batch_size

In [ ]:
# preprocess image
inputs_numpy = np.reshape(frames, (batch_size, 64, 112, 112, 3)).astype(np.float32)
inputs_numpy -= 127.5
inputs_numpy /= 127.5

In [ ]:
inputs_numpy = np.random.randn(1, 64, 112, 112, 3).astype(np.float32)
sess = rt.InferenceSession(f'{saved_model_path}/saved_model_op13_foldconst_mod_simp.onnx')  
inputs = {sess.get_inputs()[0].name: inputs_numpy}
outputs = [o.name for o in sess.get_outputs()]
outputs_onnx = sess.run(outputs, inputs)

inputs_tensor = tf.convert_to_tensor(inputs_numpy)
outputs_tensor = model(inputs_tensor)

In [ ]:
outputs_tensor[1][0][:3].numpy().T, '-'*60, outputs_onnx[1][0][:3].T

### Calculate Count

In [ ]:
raw_scores1, within_period1, sim1 = outputs_tensor[0].numpy(), outputs_tensor[1].numpy(), outputs_tensor[2].numpy()
raw_scores2, within_period2, sim2 = outputs_onnx
(raw_scores1.shape, within_period1.shape, sim1.shape), '-'*50, (raw_scores2.shape, within_period2.shape, sim2.shape)

In [ ]:
onnx_used = True
if not onnx_used:
    period_matrix = raw_scores1.reshape((-1, raw_scores1.shape[-1]))[:seq_len]
    within_period = within_period1.reshape((-1, within_period1.shape[-1]))[:seq_len]
else:
    period_matrix = raw_scores2.reshape((-1, raw_scores2.shape[-1]))[:seq_len]
    within_period = within_period2.reshape((-1, within_period2.shape[-1]))[:seq_len]
(period_matrix.shape, within_period.shape), '-'*60, \
period_matrix[0][:3], '-'*60, within_period[:3].T

####  Inv-Period Confidence Score 

In [ ]:
# Get best value from [3, 32] as the inv-period
per_frame_inv_period = np.argmax(period_matrix, axis=-1) + 1
per_frame_inv_period[-10:]

In [ ]:
inv_period_conf = np.max(softmax(period_matrix, axis=-1), axis=-1)
inv_period_conf = np.where(np.less(per_frame_inv_period, 3), 0.0, inv_period_conf)
inv_period_conf[-10:]

#### Within Period Confience Scores

In [ ]:
within_period_conf = sigmoid(within_period)[:, 0]
within_period_conf[:10]

In [ ]:
within_period_conf  = np.sqrt(np.multiply(within_period_conf, inv_period_conf))
within_period_conf[:10]

#### Period Count

In [ ]:
# median_filter
within_period_binary = np.asarray(within_period_conf > 0.5)
within_period_binary = medfilt(within_period_binary, 5)
within_period_binary[-10:]

In [ ]:
per_frame_counts = np.where(np.less(per_frame_inv_period, 3), 0.0, np.divide(1.0, per_frame_inv_period))
per_frame_counts = medfilt(per_frame_counts, 5)
per_frame_counts *= np.asarray(within_period_binary)
per_frame_counts[-10:]

In [ ]:
sum(per_frame_counts)

## TFlite

In [216]:
from tflite.BuiltinOperator import BuiltinOperator
_DIR_(BuiltinOperator)

type: <class 'type'>


'["ABS", "ADD", "ADD_N", "ARG_MAX", "ARG_MIN", "AVERAGE_POOL_2D", "BATCH_MATMUL", "BATCH_TO_SPACE_ND", "BIDIRECTIONAL_SEQUENCE_LSTM", "BIDIRECTIONAL_SEQUENCE_RNN", "CALL", "CAST", "CEIL", "CONCATENATION", "CONCAT_EMBEDDINGS", "CONV_2D", "COS", "CUMSUM", "CUSTOM", "DELEGATE", "DENSIFY", "DEPTHWISE_CONV_2D", "DEPTH_TO_SPACE", "DEQUANTIZE", "DIV", "ELU", "EMBEDDING_LOOKUP", "EMBEDDING_LOOKUP_SPARSE", "EQUAL", "EXP", "EXPAND_DIMS", "FAKE_QUANT", "FILL", "FLOOR", "FLOOR_DIV", "FLOOR_MOD", "FULLY_CONNECTED", "GATHER", "GATHER_ND", "GREATER", "GREATER_EQUAL", "HARD_SWISH", "HASHTABLE_LOOKUP", "IF", "L2_NORMALIZATION", "L2_POOL_2D", "LEAKY_RELU", "LESS", "LESS_EQUAL", "LOCAL_RESPONSE_NORMALIZATION", "LOG", "LOGICAL_AND", "LOGICAL_NOT", "LOGICAL_OR", "LOGISTIC", "LOG_SOFTMAX", "LSH_PROJECTION", "LSTM", "MATRIX_DIAG", "MATRIX_SET_DIAG", "MAXIMUM", "MAX_POOL_2D", "MEAN", "MINIMUM", "MIRROR_PAD", "MUL", "NEG", "NON_MAX_SUPPRESSION_V4", "NON_MAX_SUPPRESSION_V5", "NOT_EQUAL", "ONE_HOT", "PACK", "PAD

In [202]:
loaded_model = tf.keras.models.load_model(saved_model_path)
loaded_model.summary()

Model: "resnet_period_estimator_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_29 (Functional)        (None, None, None, 1024)  5209600   
_________________________________________________________________
conv3d_32 (Conv3D)           multiple                  14156288  
_________________________________________________________________
batch_normalization_32 (Batc multiple                  2048      
_________________________________________________________________
conv2d_29 (Conv2D)           multiple                  320       
_________________________________________________________________
dense_580 (Dense)            multiple                  1049088   
_________________________________________________________________
dense_581 (Dense)            multiple                  1049088   
_________________________________________________________________
transformer_layer_58 (Transf multiple   

In [222]:
_DIR_(tf.lite)

type: <class 'module'>


'["Interpreter", "OpsSet", "Optimize", "RepresentativeDataset", "TFLiteConverter", "TargetSpec", "experimental"]'

In [5]:
tf.__version__

'2.6.0'

In [8]:
OPT_FLOAT16 = True
# converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.allow_custom_ops = False
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    # tf.lite.OpsSet.SELECT_TF_OPS, # tflite: some op not support
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
if OPT_FLOAT16:
    converter.target_spec.supported_types = [tf.float16]
litemodel = converter.convert()

tf_model_path='%s/saved_model%s.tflite' % (saved_model_path, '_f16' if OPT_FLOAT16 else '')
with open(tf_model_path, 'wb') as f:
    f.write(litemodel)

ConverterError: <unknown>:0: error: loc(callsite(callsite("resnet_period_estimator_29/conv3d_32/Conv3D/SpaceToBatchND@__inference__wrapped_model_176814" at "StatefulPartitionedCall@__inference_signature_wrapper_194321") at "StatefulPartitionedCall")): 'tf.SpaceToBatchND' op is neither a custom op nor a flex op
<unknown>:0: note: loc("StatefulPartitionedCall"): called from
<unknown>:0: note: loc(callsite(callsite("resnet_period_estimator_29/conv3d_32/Conv3D/SpaceToBatchND@__inference__wrapped_model_176814" at "StatefulPartitionedCall@__inference_signature_wrapper_194321") at "StatefulPartitionedCall")): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(callsite("resnet_period_estimator_29/conv3d_32/Conv3D/BatchToSpaceND@__inference__wrapped_model_176814" at "StatefulPartitionedCall@__inference_signature_wrapper_194321") at "StatefulPartitionedCall")): 'tf.BatchToSpaceND' op is neither a custom op nor a flex op
<unknown>:0: note: loc("StatefulPartitionedCall"): called fro

In [163]:
_DIR_(converter)

type: <class 'tensorflow.lite.python.lite.TFLiteKerasModelConverterV2'>


'["allow_custom_ops", "convert", "experimental_enable_resource_variables", "experimental_new_converter", "experimental_new_quantizer", "inference_input_type", "inference_output_type", "optimizations", "representative_dataset", "saved_model_dir", "target_spec"]'

In [169]:
import tflite
with open(tf_model_path, 'rb') as fr:                              
    tflite_model = tflite.Model.GetRootAsModel(fr.read(), 0)

In [170]:
_DIR_(tflite_model)

type: <class 'tflite.Model.Model'>


'["Buffers", "BuffersLength", "Description", "GetRootAsModel", "Init", "Metadata", "MetadataBuffer", "MetadataBufferAsNumpy", "MetadataBufferLength", "MetadataLength", "OperatorCodes", "OperatorCodesLength", "SignatureDefs", "SignatureDefsLength", "Subgraphs", "SubgraphsLength", "Version"]'

In [175]:
tflite_model.SubgraphsLength(), tflite_model.Subgraphs(0)

(1, <tflite.SubGraph.SubGraph at 0x7f87b7f6dee8>)

In [177]:
subgraph = tflite_model.Subgraphs(0)
_DIR_(subgraph)

type: <class 'tflite.SubGraph.SubGraph'>


'["GetRootAsSubGraph", "Init", "Inputs", "InputsAsNumpy", "InputsLength", "Name", "Operators", "OperatorsLength", "Outputs", "OutputsAsNumpy", "OutputsLength", "Tensors", "TensorsLength"]'

In [212]:
print(_DIR_(subgraph.Operators(0)))
for i in range(subgraph.OperatorsLength()):
    op = subgraph.Operators(i)
    opc = tflite_model.OperatorCodes(op.OpcodeIndex())
    x = opc.CustomCode()
    if x is not None:
        print(x)
    # print(opc.CustomCode())
    # if opc.CustomCode() is not None:
    #     for j in range(op.CustomOptionsLength()):
    #         print('%03d' % j, opc.CustomOptions(j), opc.CustomCode() )

type: <class 'tflite.Operator.Operator'>
["BuiltinOptions", "BuiltinOptionsType", "CustomOptions", "CustomOptionsAsNumpy", "CustomOptionsFormat", "CustomOptionsLength", "GetRootAsOperator", "Init", "Inputs", "InputsAsNumpy", "InputsLength", "Intermediates", "IntermediatesAsNumpy", "IntermediatesLength", "MutatingVariableInputs", "MutatingVariableInputsAsNumpy", "MutatingVariableInputsLength", "OpcodeIndex", "Outputs", "OutputsAsNumpy", "OutputsLength"]
b'FlexSpaceToBatchND'
b'FlexBatchToSpaceND'
b'FlexBiasAdd'
b'FlexMul'
b'FlexAddV2'


In [172]:
for opc in tflite_model.OperatorCodes():
    print(opc)
    break

TypeError: OperatorCodes() missing 1 required positional argument: 'j'

In [146]:
interpreter = tf.lite.Interpreter(model_path=tf_model_path)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], test_inputs.shape) # modify input shape
interpreter.allocate_tensors()
input_details, output_details

([{'name': 'x',
   'index': 0,
   'shape': array([  1,  64, 112, 112,   3], dtype=int32),
   'shape_signature': array([ -1,  64, 112, 112,   3], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'Identity',
   'index': 444,
   'shape': array([ 1,  1, 32], dtype=int32),
   'shape_signature': array([-1, -1, 32], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}},
  {'name': 'Identity_1',
   'index': 568,
   'shape': array([1, 1, 1], dtype=int32),
   'shape_signature': array([-1, -1,  1], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': arra

In [148]:
interpreter.set_tensor(input_details[0]['index'], test_inputs)
interpreter.invoke()
output_data1 = interpreter.get_tensor(output_details[0]['index'])
output_data2 = interpreter.get_tensor(output_details[1]['index'])

In [149]:
output_data2[0][:10]

array([[0.671763  ],
       [0.83729875],
       [1.1195688 ],
       [1.40729   ],
       [1.6208458 ],
       [1.4918776 ],
       [1.3537045 ],
       [1.1293862 ],
       [1.4433823 ],
       [1.3766811 ]], dtype=float32)

## References

- https://github.com/onnx/onnx/blob/master/docs/Operators.md
- https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md
- https://github.com/onnx/tensorflow-onnx/blob/master/support_status.md
- https://segmentfault.com/a/1190000039936376
- https://github.com/onnx/tensorflow-onnx/issues/1606
- https://github.com/onnx/tensorflow-onnx/issues/1634
- https://chowdera.com/2020/12/20201210193138753r.html

## FAQs

- [TensorFlow Lite: TensorListFromTensor, TensorListReserve, TensorListStack, While][1]
- [Encountered unknown input type of a loop v5::Loop][2]

[1]: https://github.com/tensorflow/tensorflow/issues/33416
[2]: https://github.com/luxonis/blobconverter/issues/14